In [6]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../')
from lxrt.SlowFast.slowfast.config.defaults import get_cfg




In [7]:
from lxrt.SlowFast.slowfast.datasets.tgif_direct import TGIF

In [10]:
a = TGIF(cfg, "train")

In [9]:

cfg = get_cfg()
cfg_file = "../lxrt/SlowFast/configs/Kinetics/c2/SLOWFAST_8x8_R50.yaml"
cfg.merge_from_file(cfg_file)

In [13]:
a.__getitem__("tumblr_nkaeprvFbf1u680rpo1_400.gif")

/users/cdwivedi/RL_EXP/IDL/project/tgif-qa/code/dataset/tgif/gifs/tumblr_nqc2mbmU2J1uxhtnwo1_400.gi/*
torch.Size([3, 8, 256, 256]) torch.Size([3, 32, 256, 256])


[tensor([[[[-1.9914, -1.9918, -1.9923,  ..., -1.9882, -1.9883, -1.9884],
           [-1.9917, -1.9920, -1.9924,  ..., -1.9884, -1.9885, -1.9886],
           [-1.9919, -1.9921, -1.9924,  ..., -1.9886, -1.9888, -1.9889],
           ...,
           [-1.9930, -1.9930, -1.9930,  ..., -1.9926, -1.9927, -1.9926],
           [-1.9932, -1.9933, -1.9934,  ..., -1.9925, -1.9927, -1.9926],
           [-1.9935, -1.9936, -1.9937,  ..., -1.9926, -1.9928, -1.9926]],
 
          [[-1.9917, -1.9919, -1.9923,  ..., -1.9883, -1.9883, -1.9882],
           [-1.9919, -1.9920, -1.9922,  ..., -1.9887, -1.9886, -1.9886],
           [-1.9919, -1.9919, -1.9922,  ..., -1.9889, -1.9890, -1.9890],
           ...,
           [-1.9938, -1.9936, -1.9935,  ..., -1.9862, -1.9854, -1.9848],
           [-1.9945, -1.9942, -1.9941,  ..., -1.9865, -1.9857, -1.9850],
           [-1.9946, -1.9944, -1.9942,  ..., -1.9869, -1.9861, -1.9852]],
 
          [[-1.9915, -1.9916, -1.9919,  ..., -1.9882, -1.9882, -1.9882],
           [-

In [2]:
class TGIFDataset(Dataset):
    def __init__(self, dataset_name='train', data_type=None, dataframe_dir=None, vocab_dir=None):
        self.dataframe_dir = dataframe_dir # of the form data/tgif/vocabulary
        self.vocab_dir = vocab_dir # of the form data/tgif/dataframe
        self.data_type = data_type # 'TRANS'
        self.dataset_name = dataset_name # 'train' or 'val' or 'test'

        self.csv = self.read_from_csvfile()
        self.header2idx = self.header2idx()
        self.gif_names = self.csv[:,self.header2idx['gif_name']]
        self.gif_tensor = None
        self.questions = self.csv[:,self.header2idx['question']]
        self.answers = self.csv[:,self.header2idx['answer']]
        self.mc_options = self.csv[:,self.header2idx['a1']:header2idx['a5']+1]
        ## GIF LOADER ##
        ## NOTE: May have to change the relative path of gif dir as 
        ## an extra argument to TGIF class init
        loader  = TGIF(cfg, "train")
        self.get_gif_tensor = loader.__getitem__
        
    def __getitem__(self, i): # whats the argument for this
    	gif_path = os.path.join(self.dataframe_dir, 'gif_tensors')
    	#pick up ith gif_tensor
        #NOTE: gif_path is only the gif name, not the relative path
        # REturn value: tuple (slow frames, fast frames) where frame -> (t, 3, h, w)
        gif_tensor = self.get_gif_tensor(gif_path)
    	return self.gif_tensor, self.questions[i], self.mc_options[i], self.answers[i]

    def header2idx(self):
    	return {'gif_name':0,'question':1,'a1':2,'a2':3,'a3':4,'a4':5,'a5':6,'answer':7,'vid_id':8,'key':9}

    def read_from_csvfile(self):
        assert self.data_type in ['TRANS', 'ACTION'] # ACTION just for starting, will be using TRANS finally

        self.total_q=[]
        if self.data_type=='TRANS':
            train_data_path = os.path.join(self.dataframe_dir, 'Train_transition_question.csv')
            test_data_path = os.path.join(self.dataframe_dir, 'Test_transition_question.csv')

            
            with open(os.path.join(self.dataframe_dir, 'Total_transition_question.csv')) as file:
            	csv_reader = csv.reader(file, delimiter='\t')
            	for row in csv_reader:
            		self.total_q.append(row)

        elif self.data_type=='ACTION':
         	train_data_path = os.path.join(self.dataframe_dir, 'Train_action_question.csv')
            test_data_path = os.path.join(self.dataframe_dir, 'Test_action_question.csv')

            with open(os.path.join(self.dataframe_dir, 'Total_action_question.csv')) as file:
            	csv_reader = csv.reader(file, delimiter='\t')
            	for row in csv_reader:
            		self.total_q.append(row)
        
        self.total_q.pop(0)

        assert_exists(train_data_path)
        assert_exits(test_data_path)

        csv_data=[]
        if self.dataset_name=='train':
        	with open(train_data_path) as file:
        		csv_reader = csv.reader(file, delimiter='\t')
        		for row in csv_reader:
        			csv_data.append(row)
        elif self.dataset_name=='test':
        	with open(test_data_path) as file:
        		csv_reader = csv.reader(file, delimiter='\t')
        		for row in csv_reader:
        			csv_data.append(row)
        csv_data.pop(0)

        return np.asarray(csv_data)
    '''

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 48)

In [31]:
import os
import csv
import sys
import numpy as np
sys.path.insert(1, '../')
from lxrt.SlowFast.slowfast.config.defaults import get_cfg
from lxrt.SlowFast.slowfast.datasets.tgif_direct import TGIF


cfg = get_cfg()
cfg_file = "../lxrt/SlowFast/configs/Kinetics/c2/SLOWFAST_8x8_R50.yaml"
cfg.merge_from_file(cfg_file)

class FrameQADataset(object):
    def __init__(self, dataset_name='train', data_type=None, dataframe_dir=None, vocab_dir=None, category ="frameqa" ):
        self.dataframe_dir = dataframe_dir # of the form data/tgif/vocabulary
        self.vocab_dir = vocab_dir # of the form data/tgif/dataframe
        self.data_type = data_type # 'TRANS'
        self.dataset_name = dataset_name # 'train' or 'val' or 'test'

        self.csv, all_data = self.read_from_csvfile(category)
        self.header2idx = self.header2idx()
        self.gif_names = self.csv[:,self.header2idx['gif_name']]
        self.gif_tensor = None
        self.questions = self.csv[:,self.header2idx['question']]
        self.answer = self.csv[:,self.header2idx['answer']]
        self._build_ans_vocab(all_data[:,self.header2idx['answer']])
        ## GIF LOADER ##
        ## NOTE: May have to change the relative path of gif dir as 
        ## an extra argument to TGIF class init
        root_path = "/users/cdwivedi/RL_EXP/IDL/project/tgif-qa/code/dataset/tgif/frame_gifs/"+dataset_name+"/"
        loader  = TGIF(cfg, "train",root_path=root_path )
        self.get_gif_tensor = loader.__getitem__
        
    def _build_ans_vocab(self, all_answers):
        vocab = set()
        for ans in all_answers:
            vocab.add(str(ans))
        self.vocab = sorted(list(vocab))
        self.id2ans = self.vocab
        self.ans2id = dict(zip(self.vocab, np.arange(len(self.vocab))))
        self.vocab_len = len(self.vocab)
        self.num_answers = self.vocab_len
        self.label2ans = self.id2ans
        
    def __getitem__(self, i): # whats the argument for this
        gif_path = self.gif_names[i]
        #pick up ith gif_tensor
        #NOTE: gif_path is only the gif name, not the relative path
        # REturn value: tuple (slow frames, fast frames) where frame -> (t, 3, h, w)
        gif_tensor = self.get_gif_tensor(gif_path)
        
        return gif_tensor, self.questions[i], self.ans2id[self.answer[i]]

    def __len__(self):
        return len(self.questions)
    
    def header2idx(self):
        return {'gif_name':0,'question':1,'answer':2}

    def read_from_csvfile(self, category=None):
        print(category)
        train_data_path = os.path.join(self.dataframe_dir, 'Train_'+category+'_question.csv')
        test_data_path = os.path.join(self.dataframe_dir, 'Test_'+category+'_question.csv')
        total_data_path = os.path.join(self.dataframe_dir, 'Total_'+category+'_question.csv')
        csv_data=[]
        if self.dataset_name=='train':
            with open(train_data_path) as file:
                csv_reader = csv.reader(file, delimiter='\t')
                for row in csv_reader:
                    csv_data.append(row)
        elif self.dataset_name=='test':
            with open(test_data_path) as file:
                csv_reader = csv.reader(file, delimiter='\t')
                for row in csv_reader:
                    csv_data.append(row)
        csv_data.pop(0)
        total_csv_data=[]
        with open(total_data_path) as file:
            csv_reader = csv.reader(file, delimiter='\t')
            for row in csv_reader:
                total_csv_data.append(row)

        total_csv_data.pop(0)
        return np.asarray(csv_data), np.asarray(total_csv_data)

In [5]:
data_path= "../../../../../../IDL/project/tgif-qa/dataset/"
file = "Train_frameqa_question.csv"

In [79]:
total_q =[]
with open(data_path+file) as f:
    reader = csv.reader(f, delimiter="\t")
    for row in reader:
        total_q.append(row)

In [80]:
import numpy as np
total_q = np.asarray(total_q)

In [32]:
d = FrameQADataset(dataframe_dir="../../../../../../IDL/project/tgif-qa/dataset/", dataset_name="test")

frameqa


In [33]:
d.vocab_len

1746

In [3]:

import os
import collections
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

#from param import args
#from pretrain.qa_answer_table import load_lxmert_qa
from tasks.vqa_model import VQAModel
from tasks.vqa_data import VQADataset, VQATorchDataset, VQAEvaluator, FrameQADataset

DataTuple = collections.namedtuple("DataTuple", 'dataset loader evaluator')
from logger_utils import logger as log
logger = log("TEST1")

# def get_data_tuple(splits: str, bs:int, shuffle=False, drop_last=False) -> DataTuple:
#     dset = 7(splits)
#     tset = VQATorchDataset(dset)
#     evaluator = VQAEvaluator(dset)
#     data_loader = DataLoader(
#         tset, batch_size=bs,
#         shuffle=shuffle, num_workers=args.num_workers,
#         drop_last=drop_last, pin_memory=True
#     )

#     return DataTuple(dataset=dset, loader=data_loader, evaluator=evaluator)

def get_data_tuple(args_train, bs=32,shuffle=False, drop_last=False) -> DataTuple:
    dset = FrameQADataset(dataframe_dir="../../tgif-qa/dataset/", \
                          dataset_name="test")
    data_loader = DataLoader(
        dset, batch_size=bs,
        shuffle=shuffle, num_workers=args.num_workers,
        drop_last=drop_last, pin_memory=True
    )
    return DataTuple(dataset=dset, loader=data_loader, evaluator=None)


class VQA:
    def __init__(self):
        # Datasets
        print("Fetching data")
        self.train_tuple = get_data_tuple(
            args.train, bs=args.batch_size, shuffle=True, drop_last=True
        )
        print("Got data")
        if args.valid != "":
            self.valid_tuple = get_data_tuple(
                args.valid, bs=8,
                shuffle=False, drop_last=False
            )
        else:
            self.valid_tuple = None
        print("Got data")
        
        # Model
        print("Making model")
        self.model = VQAModel(self.train_tuple.dataset.num_answers)
        print("Ready model")
        # Print model info:
        print("Num of answers:")
        print(self.train_tuple.dataset.num_answers)
        # print("Model info:")
        # print(self.model)

        # Load pre-trained weights
        if args.load_lxmert is not None:
            self.model.lxrt_encoder.load(args.load_lxmert)
        if args.load_lxmert_qa is not None:
            load_lxmert_qa(args.load_lxmert_qa, self.model,
                           label2ans=self.train_tuple.dataset.label2ans)
        
        # GPU options
        self.model = self.model.cuda()
        if args.multiGPU:
            self.model.lxrt_encoder.multi_gpu()

        # Loss and Optimizer
        self.bce_loss = nn.BCEWithLogitsLoss()
        if 'bert' in args.optim:
            batch_per_epoch = len(self.train_tuple.loader)
            t_total = int(batch_per_epoch * args.epochs)
            print("BertAdam Total Iters: %d" % t_total)
            from lxrt.optimization import BertAdam
            self.optim = BertAdam(list(self.model.parameters()),
                                  lr=args.lr,
                                  warmup=0.1,
                                  t_total=t_total)
        else:
            self.optim = args.optimizer(self.model.parameters(), args.lr)
        
        # Output Directory
        self.output = args.output
        os.makedirs(self.output, exist_ok=True)

    def train(self, train_tuple, eval_tuple):
        dset, loader, evaluator = train_tuple
        iter_wrapper = (lambda x: tqdm(x, total=len(loader))) if args.tqdm else (lambda x: x)

        best_valid = 0.
        flag = True
        for epoch in range(args.epochs):
            quesid2ans = {}
            correct = 0
            total_loss = 0
            total = 0
            print("Len of the dataloader: ", len(loader))
#             Our new TGIFQA-Dataset returns:
#             return gif_tensor, self.questions[i], self.ans2id[self.answer[i]]
            for i, (feats1, feats2, sent, target) in iter_wrapper(enumerate(loader)):
                ques_id, boxes = -1, None
                self.model.train()
                self.optim.zero_grad()

                
                feats1, feats2, target = feats1.cuda(), feats2.cuda(), target.cuda()
                feats = [feats1, feats2]
                
                logit = self.model(feats, boxes, sent)
                assert logit.dim() == target.dim() == 2
                loss = self.bce_loss(logit, target)
                loss = loss * logit.size(1)
        
                total_loss += loss.item()
                
                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), 5.)
                self.optim.step()

                score, label = logit.max(1)
                score_t, target = target.max(1)
                correct += (label == target).sum().cpu().numpy()
                total += len(label)
                if epoch > 4:
                    for l,s,t in zip(label, sent, target):
                        print(l)
                        print(s)
                        print("Prediction", loader.dataset.label2ans[int(l.cpu().numpy())])
                        print("Answer", loader.dataset.label2ans[int(t.cpu().numpy())])
            logger.log(total_loss/len(loader), correct/len(loader)*100, epoch)
            print("=="*30)
            print("Accuracy = " , correct/total*100)
            print("Loss =" , total_loss/total)
            print("=="*30)
#             log_str = "\nEpoch %d: Train %0.2f\n" % (epoch, evaluator.evaluate(quesid2ans) * 100.)

#             if self.valid_tuple is not None:  # Do Validation
#                 valid_score = self.evaluate(eval_tuple)
#                 if valid_score > best_valid:
#                     best_valid = valid_score
#                     self.save("BEST")

#                 log_str += "Epoch %d: Valid %0.2f\n" % (epoch, valid_score * 100.) + \
#                            "Epoch %d: Best %0.2f\n" % (epoch, best_valid * 100.)

#             print(log_str, end='')

#             with open(self.output + "/log.log", 'a') as f:
#                 f.write(log_str)
#                 f.flush()

            self.save(str(epoch))

    def predict(self, eval_tuple: DataTuple, dump=None):
        """
        Predict the answers to questions in a data split.

        :param eval_tuple: The data tuple to be evaluated.
        :param dump: The path of saved file to dump results.
        :return: A dict of question_id to answer.
        """
        self.model.eval()
        dset, loader, evaluator = eval_tuple
        quesid2ans = {}
        for i, datum_tuple in enumerate(loader):
            ques_id, feats, boxes, sent = datum_tuple[:4]   # Avoid seeing ground truth
            with torch.no_grad():
                feats, boxes = feats.cuda(), boxes.cuda()
                logit = self.model(feats, boxes, sent)
                score, label = logit.max(1)
                for qid, l in zip(ques_id, label.cpu().numpy()):
                    ans = dset.label2ans[l]
                    quesid2ans[qid.item()] = ans
        if dump is not None:
            evaluator.dump_result(quesid2ans, dump)
        return quesid2ans

    def evaluate(self, eval_tuple: DataTuple, dump=None):
        """Evaluate all data in data_tuple."""
        quesid2ans = self.predict(eval_tuple, dump)
        return eval_tuple.evaluator.evaluate(quesid2ans)

    @staticmethod
    def oracle_score(data_tuple):
        dset, loader, evaluator = data_tuple
        quesid2ans = {}
        for i, (ques_id, feats, boxes, sent, target) in enumerate(loader):
            _, label = target.max(1)
            for qid, l in zip(ques_id, label.cpu().numpy()):
                ans = dset.label2ans[l]
                quesid2ans[qid.item()] = ans
        return evaluator.evaluate(quesid2ans)

    def save(self, name):
        torch.save(self.model.state_dict(),
                   os.path.join(self.output, "%s.pth" % name))

    def load(self, path):
        print("Load model from %s" % path)
        state_dict = torch.load("%s.pth" % path)
        self.model.load_state_dict(state_dict)




ModuleNotFoundError: No module named 'tasks'

In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../')
from lxrt.SlowFast.slowfast.config.defaults import get_cfg



In [9]:

from tasks.vqa_model import VQAModel


#VQAModel(1704)




usage: ipykernel_launcher.py [-h] [--train TRAIN] [--valid VALID]
                             [--test TEST] [--batchSize BATCH_SIZE]
                             [--optim OPTIM] [--lr LR] [--epochs EPOCHS]
                             [--dropout DROPOUT] [--seed SEED]
                             [--output OUTPUT] [--fast] [--tiny] [--tqdm]
                             [--load LOAD] [--loadLXMERT LOAD_LXMERT]
                             [--loadLXMERTQA LOAD_LXMERT_QA] [--fromScratch]
                             [--mceLoss] [--llayers LLAYERS]
                             [--xlayers XLAYERS] [--rlayers RLAYERS]
                             [--taskMatched] [--taskMaskLM] [--taskObjPredict]
                             [--taskQA] [--visualLosses VISUAL_LOSSES]
                             [--qaSets QA_SETS]
                             [--wordMaskRate WORD_MASK_RATE]
                             [--objMaskRate OBJ_MASK_RATE] [--multiGPU]
                             [--numWorkers NUM_W

SystemExit: 2

In [30]:
import tasks.vqa_model.VQAModel

usage: ipykernel_launcher.py [-h] [--train TRAIN] [--valid VALID]
                             [--test TEST] [--batchSize BATCH_SIZE]
                             [--optim OPTIM] [--lr LR] [--epochs EPOCHS]
                             [--dropout DROPOUT] [--seed SEED]
                             [--output OUTPUT] [--fast] [--tiny] [--tqdm]
                             [--load LOAD] [--loadLXMERT LOAD_LXMERT]
                             [--loadLXMERTQA LOAD_LXMERT_QA] [--fromScratch]
                             [--mceLoss] [--llayers LLAYERS]
                             [--xlayers XLAYERS] [--rlayers RLAYERS]
                             [--taskMatched] [--taskMaskLM] [--taskObjPredict]
                             [--taskQA] [--visualLosses VISUAL_LOSSES]
                             [--qaSets QA_SETS]
                             [--wordMaskRate WORD_MASK_RATE]
                             [--objMaskRate OBJ_MASK_RATE] [--multiGPU]
                             [--numWorkers NUM_W

SystemExit: 2

/users/cdwivedi/anaconda3/envs/project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [26]:
path = "../../snap/vqa/vqa_lxr955_tiny/9.pth"


In [27]:
state_dict = torch.load(path)
model.load_state_dict(state_dict)

In [29]:
model.lxrt_encoder.model

AttributeError: 'collections.OrderedDict' object has no attribute 'lxrt_encoder'